In [1]:
import argparse
import configparser
import time
import warnings
from distutils.util import strtobool
from json import loads
from sys import stdout

import numpy as np
from openmm import *
from openmm.app import *
from parmed.exceptions import OpenMMWarning

In [2]:
import hps

In [3]:
warnings.filterwarnings("ignore", category=OpenMMWarning)

In [4]:
config = configparser.ConfigParser(inline_comment_prefixes=("#", ";"))
config.read('md.ini')

['md.ini']

In [5]:
params = config['DEFAULT']
# Reading parameters
md_steps = int(params['md_steps'])
dt = float(params['dt']) * unit.picoseconds
nstxout = int(params['nstxout'])
nstlog = int(params['nstlog'])
model = params['model']
ref_t = float(params['ref_t']) * unit.kelvin
tau_t = float(params['tau_t']) / unit.picoseconds
pbc = strtobool(params['pbc'])
if pbc:
    box_dimension = loads(params['box_dimension'])
else:
    box_dimension = None

protein_code = params['protein_code']
checkpoint = params['checkpoint']
pdb_file = params['pdb_file']
machine = params['machine']
device = params['device']
ppn = params['ppn']
restart = strtobool(params['restart'])
if not restart:
    minimize = strtobool(params['minimize'])
else:
    minimize = False
"""
End of reading parameters
"""


'\nEnd of reading parameters\n'

In [6]:
cgModel = hps.models.buildHPSModel(pdb_file, minimize=minimize, hps_scale=model, box_dimension=box_dimension)

Generating CA hps for structure file asyn.pdb

Setting up geometrical parameters:
_________________________________
Keeping only alpha carbon atoms in topology
Added 140 CA atoms
Setting alpha-carbon masses to their average residue mass.
Setting alpha-carbon atoms radii to their statistical residue radius.
Setting alpha-carbon charge to their residue charge.
Setting hydropathy scale to their residue, Using hps_kr scale.
Added 139 bonds
Adding default bond force constant...

_________________________________
Adding Forces:
Added Harmonic Bond Forces
Added Yukawa Force
Added PairWise Force

_________________________________
Creating System Object:
Checking large bonds ...
All bonds seem to be OK (less than threshold: 0.5 nm)

_________________________________
Energy from initial structure (input structure):
The Potential Energy of the system is : -51.987186431884766 kJ/mol
The Harmonic Bond Energy is: 0.026758065447211266 kj/mol
The Yukawa Energy is: 7.3455705642700195 kj/mol
The PairWis

In [7]:
cgModel.checkBondDistances()

Checking large bonds ...
All bonds seem to be OK (less than threshold: 0.5 nm)



In [16]:
cgModel.ashbaugh_HatchForce.getGlobalParameterName(0)

'epsilon'

In [12]:
cgModel.ashbaugh_HatchForce.getGlobalParameterDefaultValue(0)

0.8368

In [13]:
cgModel.epsilon = 1

In [14]:
cgModel.ashbaugh_HatchForce.getGlobalParameterDefaultValue(0)

0.8368

In [17]:
cgModel.ashbaugh_HatchForce.setGlobalParameterDefaultValue(0,2)

In [18]:
cgModel.ashbaugh_HatchForce.getGlobalParameterDefaultValue(0)

2.0